In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [24]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [6]:
train_datagen = ImageDataGenerator(
                                rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True
                    )
training_set = train_datagen.flow_from_directory(
                                '../input/pneumonia/New folder (2)/train',
                                batch_size=32,
                                target_size=(64,64),
                                class_mode='binary'
                    )

In [7]:
val_datagen = ImageDataGenerator(
                                rescale=1./255
                    )
val_set = train_datagen.flow_from_directory(
                                '../input/pneumonia/New folder (2)/valid',
                                batch_size=32,
                                target_size=(64,64),
                                class_mode='binary'
                    )

In [8]:
test_datagen = ImageDataGenerator(
                                rescale=1./255
                    )
test_set = train_datagen.flow_from_directory(
                                '../input/pneumonia/New folder (2)/test',
                                batch_size=32,
                                target_size=(64,64),
                                class_mode='binary'
                    )

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D

In [11]:
cnn = Sequential()

In [13]:
cnn.add(Conv2D(filters=100,kernel_size=3,activation='relu',input_shape=(64,64,3)))
cnn.add(MaxPooling2D(pool_size=2,strides=2))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(filters=200,kernel_size=3,activation='relu'))
cnn.add(MaxPooling2D(pool_size=2,strides=2))
cnn.add(Dropout(0.3))
cnn.add(Conv2D(filters=200,kernel_size=3,activation='relu'))
cnn.add(MaxPooling2D(pool_size=2,strides=2))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(filters=400,kernel_size=3,activation='relu'))
cnn.add(Dropout(0.3))
cnn.add(Flatten())
cnn.add(Dense(units=128,activation='relu'))
cnn.add(Dropout(0.2))
cnn.add(Dense(units=1,activation='sigmoid'))

In [14]:
cnn.summary()

In [15]:
cnn.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
r = cnn.fit(x=training_set,validation_data=val_set,epochs=15)

In [18]:
plt.figure(figsize=(12,8))
plt.plot(r.history['loss'],label='train loss')
plt.plot(r.history['val_loss'],label='validation loss')
plt.legend();

In [19]:
plt.figure(figsize=(12,8))
plt.plot(r.history['accuracy'],label='train accuracy')
plt.plot(r.history['val_accuracy'],label='validation accuracy')
plt.legend();

In [21]:
test_predictions = cnn.predict(test_set)
pred = []

for x in test_predictions:
    if x > 0.5:
        pred.append(1)
    else:
        pred.append(0)

In [36]:
ax = sns.countplot(pred)
ax.set_xticklabels(['NORMAL','PNEUMONIA'])
for p in ax.patches:
    ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.32, p.get_height()+1));

Therefore, on the test set, there are a total of 438 pneumonia cases.